In [1]:
#Import dependancies
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') #Encoder text to tensor
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)


c:\Users\Christian Gauthier\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [3]:
#Get Yelp dataset and standardise

reviews = pd.read_csv('reviews.csv')
reviews = shuffle(reviews)

#Standardise and tokenize
for column in reviews:
    reviews['text'] = reviews['text'].str.lower()   #Covert the text to lower case
    reviews['text'].str.replace('[^\w\s]','') #Remove punctuation
    reviews['text'].str.strip() #Remove whitespace
    reviews['text'].str.replace("\n", " ") #Remove escape characters
reviews = reviews['text']

#Get the max token from training data to be used in model
max_length = 0
for row in reviews:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))

print('max length: ' + str(max_length))

reviews = reviews[:5000]

print(reviews)

C:\Users\Christian Gauthier\AppData\Local\Temp\ipykernel_22156\3893656625.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['text'].str.replace('[^\w\s]','') #Remove punctuation


max length: 1006
57104    this was one of my boyfriend's favorite places...
65110    birchtree catered and pretty much helped plan ...
32763    ray's is my favorite place to get subs in nj. ...
36231    i booked an appointment this afternoon for bot...
85442    the restaurant is small so it produced a long ...
                               ...                        
27292    loved this place. after an unpleasant experien...
80798    on top of it like piraat ale. great grub. bone...
63060    so a friend and i went to this pakistani "hole...
88494    good pad thai and spring rolls. fast service e...
50632    so sick!!!  staff was terrible and the produce...
Name: text, Length: 5000, dtype: object


C:\Users\Christian Gauthier\AppData\Local\Temp\ipykernel_22156\3893656625.py:22: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  reviews = reviews[:5000]


In [3]:
input_ids = tokenizer.encode(reviews.iloc[1], return_tensors='tf')


In [4]:
single_string = ''
for row in reviews:
  x = row
  single_string += x 
string_tokenized = tokenizer.encode(single_string)

Token indices sequence length is longer than the specified maximum sequence length for this model (653274 > 1024). Running this sequence through the model will result in indexing errors


In [7]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [6]:
# defining optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [8]:
history = model.fit(dataset, epochs=25)

Epoch 1/25
544/544 [==============================] - 136s 228ms/step - loss: 3.7692 - accuracy: 0.3066
Epoch 2/25
544/544 [==============================] - 124s 229ms/step - loss: 3.5566 - accuracy: 0.3270
Epoch 3/25
544/544 [==============================] - 125s 229ms/step - loss: 3.4539 - accuracy: 0.3374
Epoch 4/25
544/544 [==============================] - 125s 230ms/step - loss: 3.3672 - accuracy: 0.3472
Epoch 5/25
544/544 [==============================] - 125s 229ms/step - loss: 3.2917 - accuracy: 0.3551
Epoch 6/25
544/544 [==============================] - 125s 229ms/step - loss: 3.2212 - accuracy: 0.3629
Epoch 7/25
544/544 [==============================] - 125s 229ms/step - loss: 3.1513 - accuracy: 0.3713
Epoch 8/25
544/544 [==============================] - 121s 222ms/step - loss: 3.0844 - accuracy: 0.3793
Epoch 9/25
544/544 [==============================] - 119s 219ms/step - loss: 3.0197 - accuracy: 0.3867
Epoch 10/25
544/544 [==============================] - 119s 219m

In [14]:
text = "For dinner,"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
output = model.generate(
  input_ids,
  max_length = 150,
  num_beams = 5,
  temperature = 0.8,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(output[0]))

For dinner, my wife and i split an appetizer and two entrees. we split the crab cakes and macaroni and cheese. both were very good. they were fresh and had a nice balance of flavor. the macarons were also very tasty. i would definitely come back for more. 
the service was great. our waiter was very attentive and took the time to check on us and make sure we were enjoying the food we ordered. it was a very pleasant experience and we look forward to trying everything else on their menu!i love this place! i've been going here for years and it's one of my favorite places in town! the staff is always friendly and the prices are very reasonable! they have a wide


In [12]:
text = "The best part was"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
output = model.generate(
  input_ids,
  max_length = 150,
  num_beams = 5,
  temperature = 0.85,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(output[0]))

The best part was the service. we were seated quickly and the place was clean. the staff was friendly and attentive. i would highly recommend this restaurant.i love this place! i've been coming here for years and it's always a good experience. they have a great variety of sandwiches, pastas, and pastries. my favorite is the sweet potato pastrami and my all time favorite the chicken and waffles. 
the only downside i can think of is that they don't always have happy hour so make sure you go during the week in order to save a few bucks. but i will definitely come back for sure!i'm a big fan of la colombe burrito and this is by far the best bur


In [15]:
text = "Every time we go,"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
output = model.generate(
  input_ids,
  max_length = 100,
  num_beams = 5,
  temperature = 0.85,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(output[0]))

Every time we go, we are greeted with a warm smile and a knowing nod from our waiter.  it is a great way to give back to our community and show that we care about our food and are not afraid to try something new.i love this place. i've been going here for years and i can't say enough good things about the staff. they are always friendly and the food is always great. the only downside is the price. it's a little expensive for what you get


In [16]:
text = "Their menu changes"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
output = model.generate(
  input_ids,
  max_length = 125,
  num_beams = 5,
  temperature = 0.92,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(output[0]))

Their menu changes all the time, so it's hard to know what to expect.  i'm not sure if we'll be back next year, but we will definitely stop in for the oysters and the crawfish bisque.my husband and i have been going to this place since we were little. we love it. the staff is always friendly and helpful. they have a wide variety of specialty pizzas to choose from. i always get the calabacitas and they are delicious. it is a little pricey for what you get but the portions are very generous. if you are looking for something different then this


In [17]:
text = "Our waitress"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
output = model.generate(
  input_ids,
  max_length = 200,
  num_beams = 5,
  temperature = 0.845,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(output[0]))

Our waitress was very nice and attentive.  i ordered the shrimp and grits and it was good but the sauce was too sweet for my taste. i will definitely go back to try other dishes on the menu.this is a great place to get a quick bite to eat while in the tampa bay area. they have a wide variety of sandwiches, salads, and smoothies to choose from. the staff is always friendly and always willing to help you get the most out of your visit.my husband and i have been going to this place since we were little. we love it. it's tucked away in a shopping center right by the airport. you can walk in and get free chips and salsa to dip into the delicious salsa bar. service is quick and friendly. food is fresh and delicious. my husband loves his burritos and queso burrito. their tortillas are huge and are made to order. this is one of our favorite places to stop for lunch or dinner.


In [18]:
text = "When I walked in,"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
output = model.generate(
  input_ids,
  max_length = 250,
  num_beams = 5,
  temperature = 0.82,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(output[0]))

When I walked in, it was already dark and dingy with only a few tables in the place.  i walked out and asked the host if there was anything else they could do to make my meal better, and he said he would look into it, but that he couldn't do anything since they were closing in on a saturday night.
i ordered the chicken and waffles, which were good but nothing to write home about. the waffle was so dense and dense that i had to eat it with a fork and knife. when i got home and looked for the kitchen where the food was being prepared, i realized there wasn't one in there. instead, there were two dirty silverware stacked on the counter. one was dripping wet with grease, one of which had been sitting in my food. i could only see the grease dripping down the side of the panini bread with my fork. it looked like it had come from the sink or some other dirty sink fixture. my order of fries had grease all over them, including the ones with the lollipop in them. they tasted like they came from 